# Pandas урок

---

### Завдання:
<a name="0"/>

[1. Завантаження данних](#1)<br>

[2. Аналіз та спрощення данних](#2)<br>

[3. Скільки чоловіків і жінок (ознака `sex`) представлено в цьому наборі даних?](#3)<br>

[4. Який середній вік (ознака `age`) жінок?](#4)<br>

[5. Яка частка громадян Німеччини (ознака `native-country`)](#5)<br>

[6. Які середні значення і середньоквадратичні відхилення віку тих,<br>
хто отримує понад 50K на рік (ознака `salary`), і тих, хто отримує менш як 50K на рік??](#6)<br>

[7. Чи правда, що люди, які отримують понад 50k, мають щонайменше вищу освіту?<br>
(ознака `education - Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters або Doctorate`)](#7)<br>

[8. Виведіть статистику віку для кожної раси (ознака `race`) і кожної статі.<br>
Використовуйте `groupby` та `describe`. Знайдіть таким чином максимальний вік<br>
чоловіків раси `Amer-Indian-Eskimo`](#8)<br>

[9. Серед кого більша частка тих, хто заробляє багато (>50K):<br>
серед одружених чи неодружених чоловіків (ознака `marital-status`)?<br>
Одруженими вважаємо тих, у кого `marital-status` починається з `Married`<br>
(`Married-civ-spouse, Married-spouse-absent або Married-AF-spouse`),<br>
решту вважаємо неодруженими.](#9)<br>

[10. Яке максимальне число годин людина працює на тиждень (ознака `hours-per-week`)?<br>
Скільки людей працюють таку кількість годин і який серед них відсоток тих, хто заробляє багато?](#10)<br>

[11. Порахуйте середній час роботи (`hours-per-week`) тих,<br>
хто заробляє мало і багато (`salary`) для кожної країни (`native-country`)](#11)<br>

Імпорт бібліотек

In [138]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

import pandas as pd
import numpy as np

---

# Рішення

---

#### 1. Завантаження данних
[завдання](#0)
<a name="1"/>


In [139]:
# Посилання на архів
url = 'https://archive.ics.uci.edu/static/public/2/adult.zip'

Мною були помічені відступи у текстових фічах при звичайному іморті. Тому додамо функцію попередьної очистки.

In [140]:
def strip(text):
    try:
        return text.strip()
    except AttributeError:
        return text

In [141]:
resp = urlopen(url)
myzip = ZipFile(BytesIO(resp.read()))
with myzip.open('adult.data') as myfile:
    #print(myfile.readline())
    data = pd.read_csv(myfile, names=['age','workclass', 'fnlwgt', 'education', 'education-num',
                                      'marital-status', 'occupation', 'relationship', 'race',
                                      'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
                                      'native-country', 'salary'],
                      converters = {'workclass' : strip,
                                    'education' : strip,
                                    'marital-status' : strip,
                                    'occupation' : strip,
                                    'relationship' : strip,
                                    'race' : strip,
                                    'sex' : strip,
                                    'native-country' : strip,
                                    'salary' : strip})

data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


---

#### 2. Аналіз та спрощення данних
[завдання](#0)
<a name="2"/>

In [142]:
# Інформація про датасет
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  salary          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


Типи даних для фічей можна змінити, спростивши таблицю, зробивши її меншою за вагою і прискоривши процес обробки.

---

Перевіримо min, max значення за int'овими стовпцями:

In [143]:
data.describe(include=['int64'])

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


Ось таблиця типів даних і скільки символів вони можуть містити:<br>
*    **int16**	Integer (-32768 to 32767)<br>
*    **int32**	Integer (-2147483648 to 2147483647)

---

Змінимо `age`, `education-num`, `capital-loss`, `hours-per-week` на **int16**, а `fnlwgt`
та `capital-gain` на **int32**

In [144]:
data['age'] = data['age'].astype('int16')
data['education-num'] = data['education-num'].astype('int16')
data['capital-loss'] = data['capital-loss'].astype('int16')
data['hours-per-week'] = data['hours-per-week'].astype('int16')
data['fnlwgt'] = data['fnlwgt'].astype('int32')
data['capital-gain'] = data['capital-gain'].astype('int32')

In [145]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int16 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int32 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int16 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int32 
 11  capital-loss    32561 non-null  int16 
 12  hours-per-week  32561 non-null  int16 
 13  native-country  32561 non-null  object
 14  salary          32561 non-null  object
dtypes: int16(4), int32(2), object(9)
memory usage: 2.7+ MB


Дані типу **'object'** `workclass`, `education`, `marital-status`, `occupation`, `relationship`, `race`, `sex`, `native-country` є категоріями(`category`).

---

Змінимо ці типи даних.
Хоч я і розумію, що стовпчик sex має лише 2 значення (`Male, Female`) і теж може бути змінений як **bool**, але не знаю, чи правильно така зміна з етичного погляду, тому залишу категоріальним.

In [146]:
data['workclass'] = data['workclass'].astype('category')
data['education'] = data['education'].astype('category')
data['marital-status'] = data['marital-status'].astype('category')
data['occupation'] = data['occupation'].astype('category')
data['relationship'] = data['relationship'].astype('category')
data['race'] = data['race'].astype('category')
data['sex'] = data['sex'].astype('category')
data['native-country'] = data['native-country'].astype('category')

In [147]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   age             32561 non-null  int16   
 1   workclass       32561 non-null  category
 2   fnlwgt          32561 non-null  int32   
 3   education       32561 non-null  category
 4   education-num   32561 non-null  int16   
 5   marital-status  32561 non-null  category
 6   occupation      32561 non-null  category
 7   relationship    32561 non-null  category
 8   race            32561 non-null  category
 9   sex             32561 non-null  category
 10  capital-gain    32561 non-null  int32   
 11  capital-loss    32561 non-null  int16   
 12  hours-per-week  32561 non-null  int16   
 13  native-country  32561 non-null  category
 14  salary          32561 non-null  object  
dtypes: category(8), int16(4), int32(2), object(1)
memory usage: 1021.6+ KB


---

Перевіримо стовпець `salary`. Якщо там тільки 2 значення (salary: '<=50К', '>50K'), то замінимо його на тип `bool`, де (selary: '>50К'==True)

In [148]:
data['salary'].unique()

array(['<=50K', '>50K'], dtype=object)

In [149]:
data['salary'] = data['salary'].map({'>50K': True, '<=50K': False}).astype('bool')

Оскільки далі буде не дуже зрозумілим значення (`True, False`) для стовпця `salary` - перейменуємо його на `salary >50K`

In [150]:
data = data.rename(columns={"salary": "salary >50K"})
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary >50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,False
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,False
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,False
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,False
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,False


In [151]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   age             32561 non-null  int16   
 1   workclass       32561 non-null  category
 2   fnlwgt          32561 non-null  int32   
 3   education       32561 non-null  category
 4   education-num   32561 non-null  int16   
 5   marital-status  32561 non-null  category
 6   occupation      32561 non-null  category
 7   relationship    32561 non-null  category
 8   race            32561 non-null  category
 9   sex             32561 non-null  category
 10  capital-gain    32561 non-null  int32   
 11  capital-loss    32561 non-null  int16   
 12  hours-per-week  32561 non-null  int16   
 13  native-country  32561 non-null  category
 14  salary >50K     32561 non-null  bool    
dtypes: bool(1), category(8), int16(4), int32(2)
memory usage: 799.0 KB


Отже, вдалося зменшити розмір датасету з `3.7Мб` до `0.8Мб`

---

Перевіремо значення категоріальних фіч і їх кількість

In [152]:
data['workclass'].value_counts(dropna=False)

Private             22696
Self-emp-not-inc     2541
Local-gov            2093
?                    1836
State-gov            1298
Self-emp-inc         1116
Federal-gov           960
Without-pay            14
Never-worked            7
Name: workclass, dtype: int64

**Як бачимо є 1836 невизначених(?) значень**

In [153]:
data['education'].value_counts(dropna=False)

HS-grad         10501
Some-college     7291
Bachelors        5355
Masters          1723
Assoc-voc        1382
11th             1175
Assoc-acdm       1067
10th              933
7th-8th           646
Prof-school       576
9th               514
12th              433
Doctorate         413
5th-6th           333
1st-4th           168
Preschool          51
Name: education, dtype: int64

In [154]:
data['marital-status'].value_counts(dropna=False)

Married-civ-spouse       14976
Never-married            10683
Divorced                  4443
Separated                 1025
Widowed                    993
Married-spouse-absent      418
Married-AF-spouse           23
Name: marital-status, dtype: int64

In [155]:
data['occupation'].value_counts(dropna=False)

Prof-specialty       4140
Craft-repair         4099
Exec-managerial      4066
Adm-clerical         3770
Sales                3650
Other-service        3295
Machine-op-inspct    2002
?                    1843
Transport-moving     1597
Handlers-cleaners    1370
Farming-fishing       994
Tech-support          928
Protective-serv       649
Priv-house-serv       149
Armed-Forces            9
Name: occupation, dtype: int64

**Тут теж є 1843 невизначених(?) значень**

In [156]:
data['relationship'].value_counts(dropna=False)

Husband           13193
Not-in-family      8305
Own-child          5068
Unmarried          3446
Wife               1568
Other-relative      981
Name: relationship, dtype: int64

In [157]:
data['race'].value_counts(dropna=False)

White                 27816
Black                  3124
Asian-Pac-Islander     1039
Amer-Indian-Eskimo      311
Other                   271
Name: race, dtype: int64

In [158]:
data['native-country'].value_counts(dropna=False)

United-States                 29170
Mexico                          643
?                               583
Philippines                     198
Germany                         137
Canada                          121
Puerto-Rico                     114
El-Salvador                     106
India                           100
Cuba                             95
England                          90
Jamaica                          81
South                            80
China                            75
Italy                            73
Dominican-Republic               70
Vietnam                          67
Guatemala                        64
Japan                            62
Poland                           60
Columbia                         59
Taiwan                           51
Haiti                            44
Iran                             43
Portugal                         37
Nicaragua                        34
Peru                             31
Greece                      

**І тут 583 невизначених(?) значень**

Перевіремо в таблиці на рядки (`workclass`, `occupation`, `native-country`). Це просто одне пропущене значення в рядку, або весь рядок порожній

In [159]:
miss_data = data[(data['workclass']=='?') & (data['occupation']=='?') & (data['native-country']=='?')]
miss_data.shape

(27, 15)

Є невелика кількість рядків, у яких пропущено всі три значення. Однак більшість розкидані по інших рядках. Під час подальшого аналізу потрібно бути уважним під час вибору досліджуваних фічей, і фільтрувати їх перед аналізом.

---

#### 3. Скільки чоловіків і жінок (ознака sex) представлено в цьому наборі даних?
[завдання](#0)
<a name="3"/>

In [160]:
sex_age = data['sex'].value_counts(dropna=True)
print(f'''Чоловіків: {sex_age[0]}
Жінок: {sex_age[1]}''')

Чоловіків: 21790
Жінок: 10771


---

#### 4. Який середній вік (ознака age) жінок??
[завдання](#0)
<a name="4"/>

In [161]:
print(f'''Середній вік жінок: {data[data['sex']=='Female']['age'].mean()} роки''')


Середній вік жінок: 36.85823043357163 роки


---

#### 5. Яка частка громадян Німеччини (ознака native-country)?
[завдання](#0)
<a name="5"/>

In [162]:
print(f'''Частка громадян Німеччини
{data[data['native-country']=='Germany']['native-country'].count() / data[data['native-country'] != '?']['native-country'].count() * 100}%''')

Частка громадян Німеччини
0.42841953843267244%


---

#### 6. Які середні значення і середньоквадратичні відхилення віку тих, хто отримує понад 50K на рік (ознака salary), і тих, хто отримує менш як 50K на рік??
[завдання](#0)
<a name="6"/>

In [163]:
salary_age = data[data['salary >50K']==True]['age']
print(f'''Для тих хто отримує: >50K
Середнє значення віку: {salary_age.mean()}
Середньоквадратичне відхилення віку: {salary_age.std()}''')

Для тих хто отримує: >50K
Середнє значення віку: 44.24984058155847
Середньоквадратичне відхилення віку: 10.519027719851826


---

#### 7. Чи правда, що люди, які отримують понад 50k, мають щонайменше вищу освіту? (ознака education - Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters або Doctorate)
[завдання](#0)
<a name="7"/>

In [164]:
higher_education = ['Bachelors', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', 'Masters', 'Doctorate']
data[data['salary >50K']==True]['education'].map(lambda education: education in higher_education).value_counts(dropna=False)

True     4535
False    3306
Name: education, dtype: int64

Не правда. як бачимо близько 40% людей, що отримують >50K не мають вищу освіту 

---

#### 8. Виведіть статистику віку для кожної раси (ознака race) і кожної статі. Використовуйте groupby та describe. Знайдіть таким чином максимальний вік чоловіків раси Amer-Indian-Eskimo.
[завдання](#0)
<a name="8"/>

In [165]:
data.groupby(['race', 'sex'])['age'].describe()

count       mean        std   min   25%   50%  \
race               sex                                                       
Amer-Indian-Eskimo Female    119.0  37.117647  13.114991  17.0  27.0  36.0   
                   Male      192.0  37.208333  12.049563  17.0  28.0  35.0   
Asian-Pac-Islander Female    346.0  35.089595  12.300845  17.0  25.0  33.0   
                   Male      693.0  39.073593  12.883944  18.0  29.0  37.0   
Black              Female   1555.0  37.854019  12.637197  17.0  28.0  37.0   
                   Male     1569.0  37.682600  12.882612  17.0  27.0  36.0   
Other              Female    109.0  31.678899  11.631599  17.0  23.0  29.0   
                   Male      162.0  34.654321  11.355531  17.0  26.0  32.0   
White              Female   8642.0  36.811618  14.329093  17.0  25.0  35.0   
                   Male    19174.0  39.652498  13.436029  17.0  29.0  38.0   

                             75%   max  
race               sex                  
Amer-Indian-Eskimo Female  46.00  80.0  
                   Male    45.00  82.0  
Asian-Pac-Islander Female  43.75  75.0  
                   Male    46.00  90.0  
Black              Female  46.00  90.0  
                   Male    46.00  90.0  
Other              Female  39.00  74.0  
                   Male    42.00  77.0  
White              Female  46.00  90.0  
                   Male    49.00  90.0

In [166]:
print(f'''Cередній вік: чоловіка
Раси: Amer-Indian-Eskimo
{data[(data['race']=='Amer-Indian-Eskimo') & (data['sex']=='Male')]['age'].max()} роки''')

Cередній вік: чоловіка
Раси: Amer-Indian-Eskimo
82 роки


---

#### 9. Серед кого більша частка тих, хто заробляє багато (>50K): серед одружених чи неодружених чоловіків (ознака marital-status)? Одруженими вважаємо тих, у кого marital-status починається з Married (Married-civ-spouse, Married-spouse-absent або Married-AF-spouse), решту вважаємо неодруженими.
[завдання](#0)
<a name="9"/>

In [167]:
marital_statueses = ['Married-civ-spouse', 'Married-spouse-absent', 'Married-AF-spouse']
a, b = data[(data['salary >50K'] == True) & (data['sex'] == 'Male')]['marital-status'].map(lambda ms: ms in marital_statueses).value_counts(dropna=True)
print(f'''
Для:                Чоловіків
Із заробітком:      >50K
Частка одружених:   {a/(a+b)}
Частка неодружених: {b/(a+b)}''')


Для:                Чоловіків
Із заробітком:      >50K
Частка одружених:   0.8953767637346143
Частка неодружених: 0.10462323626538576


---

#### 10. Яке максимальне число годин людина працює на тиждень (ознака hours-per-week)? Скільки людей працюють таку кількість годин і який серед них відсоток тих, хто заробляє багато?
[завдання](#0)
<a name="10"/>

In [168]:
max_work = data['hours-per-week'].max()
max_work_count = data[(data['hours-per-week'] == max_work)]['hours-per-week'].count()
max_work_max_salary = data[(data['hours-per-week'] == max_work) & (data['salary >50K'] == True)]['salary >50K'].count()

print(f'''
Max число годин на тиждень:       {max_work}
Людей, які працюють max годин:    {max_work_count}
Відсоток людей що отримують >50K: {max_work_max_salary/max_work_count*100} %''')


Max число годин на тиждень:       99
Людей, які працюють max годин:    85
Відсоток людей що отримують >50K: 29.411764705882355 %


---

#### 11. Порахуйте середній час роботи (hours-per-week) тих, хто заробляє мало і багато (salary) для кожної країни (native-country).
[завдання](#0)
<a name="11"/>

In [169]:
data.groupby(['salary >50K', 'native-country'])['hours-per-week'].mean()

salary >50K  native-country 
False        ?                  40.164760
             Cambodia           41.416667
             Canada             37.914634
             China              37.381818
             Columbia           38.684211
                                  ...    
True         Thailand           58.333333
             Trinadad&Tobago    40.000000
             United-States      45.505369
             Vietnam            39.200000
             Yugoslavia         49.500000
Name: hours-per-week, Length: 84, dtype: float64

### Дякую за увагу!)